# Coupled higgs equation 
$$
u_{tt} - u_{xx} + |u|^2 u - 2uv = 0
$$
$$
v_{tt} + v_{xx} - (\left| u \right|^2)_{xx} = 0
$$

where, $ u(x,t) $ represents a complex nucleon field and $ v(x,t) $ represents a real scalar meson field. The coupled Higgs field Equation describes a system of conserved scalar nucleon interaction with a neutral scalar meson.

solutions 

$$
u_1(x, t) = ir e^{ir(\omega x + t)} \sqrt{1 + \omega^2} \tanh\left(\frac{r(k + x + \omega t)}{\sqrt{2}}\right)
$$
$$
v_1(x, t) = r^2 \tanh^2\left(\frac{r(k + x + \omega t)}{\sqrt{2}}\right)
$$

for $t = 0$

$$
u_1(x, 0) = ir e^{ir \omega x} \sqrt{1 + \omega^2} \tanh\left(\frac{r(k + x)}{\sqrt{2}}\right)
$$
$$
v_1(x, 0) = r^2 \tanh^2\left(\frac{r(k + x)}{\sqrt{2}}\right)
$$

where 
$k = 4, \omega = 5 , \alpha = 2, c = 2, r = 2$


In [9]:
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.autograd import Variable
from torch.optim import LBFGS
import torch.nn.functional as F
from tqdm import tqdm
import numpy as np
import os
import pandas as pd

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [10]:
import torch
import torch.nn as nn

# Base class for shared functionalities
class BaseNN(nn.Module):
    def __init__(self):
        super(BaseNN, self).__init__()

    def _make_layers(self, in_features, out_features, num_layers, activation):
        layers = [nn.Linear(in_features, out_features), activation()]
        for _ in range(1, num_layers):
            layers += [nn.Linear(out_features, out_features), activation()]
        return nn.Sequential(*layers)

# Model for U1 Real and U1 Imaginary
class ModelU1(BaseNN):
    def __init__(self, input_dim=2, units=64, output_dim=1, layers=3, activation=nn.Tanh):
        super(ModelU1, self).__init__()
        self.layers = self._make_layers(input_dim, units, layers, activation)
        self.final = nn.Linear(units, output_dim)

    def forward(self, x):
        return self.final(self.layers(x))

# Model for V1 with potential adaptations for functions like sinc
class ModelV1(BaseNN):
    def __init__(self, input_dim=2, units=64, output_dim=1, layers=2, activation=nn.Tanh):
        super(ModelV1, self).__init__()
        # Possible adjustment for sinc-like behavior could be modifying the layer depth or activation functions
        self.layers = self._make_layers(input_dim, units, layers, activation)
        self.final = nn.Linear(units, output_dim)

    def forward(self, x):
        return self.final(self.layers(x))


In [11]:
def grad(x, t):
    return torch.autograd.grad(x, t, grad_outputs=torch.ones_like(x), create_graph=True, retain_graph=True)[0]

def laplacian(field, x, t):
    field_x = grad(field, x)
    field_xx = grad(field_x, x)
    field_t = grad(field, t)
    field_tt = grad(field_t, t)
    return field_xx, field_tt

# Define the ODE system for the Coupled Higgs field equations
def coupled_higgs(u_real, u_imag, v, x, t):
    u_r_xx, u_r_tt = laplacian(u_real, x, t)
    u_i_xx, u_i_tt = laplacian(u_imag, x, t)
    v_xx, v_tt = laplacian(v, x, t)

    # Calculate |u|^2 = u_real^2 + u_imag^2 and its second derivative w.r.t x
    u_magnitude_squared = torch.square(u_real) + torch.square(u_imag)
    u_magnitude_squared_xx, _ = laplacian(u_magnitude_squared, x, t)
    
    # Calculate residuals based on the differential equations
    residual_u_real = u_r_tt - u_r_xx + u_magnitude_squared * u_real - 2 * u_real * v
    residual_u_imag = u_i_tt - u_i_xx + u_magnitude_squared * u_imag - 2 * u_imag * v
    residual_v = v_tt + v_xx - u_magnitude_squared_xx

    return residual_u_real, residual_u_imag, residual_v

# Function to calculate the real part of u1
def real_u1(x, t, k, omega, r):
    complex_exp = torch.exp(1j * r * (omega * x + t))
    tanh_val = torch.tanh((r * (k + x + omega * t)) / torch.sqrt(torch.tensor(2.0)))
    result = torch.real(1j * r * complex_exp * torch.sqrt(torch.tensor(1) + omega**2) * tanh_val)
    return result

def imag_u1(x, t, k, omega, r):
    complex_exp = torch.exp(1j * r * (omega * x + t))
    tanh_val = torch.tanh((r * (k + x + omega * t)) / torch.sqrt(torch.tensor(2.0)))
    result = torch.imag(1j * r * complex_exp * torch.sqrt(torch.tensor(1) + omega**2) * tanh_val)
    return result

def real_v1(x, t, k, omega, r):
    result = (r * torch.tanh((r * (k + x + omega * t)) / torch.sqrt(torch.tensor(2.0))))**2
    return result

def compute_analytical_boundary_ur_loss(model, x_boundary, t_boundary, mse_cost_function, k, omega, r):
    x_t_boundary = torch.cat([x_boundary, t_boundary], dim=1)     
    pred_u_r = model(x_t_boundary)
    real_u1_val = real_u1(x_boundary, t_boundary, k, omega, r)
    boundary_loss_ur = mse_cost_function(pred_u_r, real_u1_val)
    return boundary_loss_ur

def compute_analytical_boundary_ui_loss(model, x_boundary, t_boundary, mse_cost_function, k, omega, r):
    x_t_boundary = torch.cat([x_boundary, t_boundary], dim=1)     
    pred_u_i = model(x_t_boundary)
    imag_u1_val = imag_u1(x_boundary, t_boundary, k, omega, r)
    boundary_loss_ui = mse_cost_function(pred_u_i, imag_u1_val)
    return boundary_loss_ui

def compute_analytical_boundary_v_loss(model, x_boundary, t_boundary, mse_cost_function, k, omega, r):
    x_t_boundary = torch.cat([x_boundary, t_boundary], dim=1)     
    pred_v  = model(x_t_boundary)
    real_v1_val = real_v1(x_boundary, t_boundary, k, omega, r)
    boundary_loss_v = mse_cost_function(pred_v, real_v1_val)
    
    return boundary_loss_v

def compute_physics_loss(model1, model2, model3, x, t, mse_cost_function):
    x.requires_grad = True
    t.requires_grad = True
    x_t = torch.cat([x, t], dim=1) 
    pred_u_r = model1(x_t)
    pred_u_i = model2(x_t)
    pred_v   = model3(x_t)
    
    du_eq_r, du_eq_i, dv_eq = coupled_higgs(pred_u_r, pred_u_i, pred_v, x, t)
    
    zeros_r = torch.zeros_like(du_eq_r, device=device)
    zeros_i = torch.zeros_like(du_eq_i, device=device)
    zeros_v = torch.zeros_like(dv_eq, device=device)
    
    # Compute the MSE loss against zeros for each differential equation residual
    loss_ur = mse_cost_function(du_eq_r, zeros_r)
    loss_ui = mse_cost_function(du_eq_i, zeros_i)
    loss_v = mse_cost_function(dv_eq, zeros_v)
    
    return loss_ur, loss_ui, loss_v

In [12]:
# Check if CUDA is available and set the default device
if torch.cuda.is_available():
    print("CUDA is available! Training on GPU.")
else:
    print("CUDA is not available. Training on CPU.")

u1_real_model = ModelU1(input_dim=2, units=64, output_dim=1, layers=3, activation=nn.Tanh)
u1_imag_model = ModelU1(input_dim=2, units=64, output_dim=1, layers=3, activation=nn.Tanh)
v1_model = ModelV1(input_dim=2, units=64, output_dim=1, layers=3, activation=nn.Tanh)

num_epochs = 100000  # Number of training epochs
lr = 1e-3          # Learning rate
num_samples = 1000 # Number of samples for training
r = 1.1
omega = 2
k = 0.5
factor = 1
lambda_ = 1e-3

optimizer_u1_real = Adam(u1_real_model.parameters(), lr=lr)
optimizer_u1_imag = Adam(u1_imag_model.parameters(), lr=lr)
optimizer_v1 = Adam(v1_model.parameters(), lr=lr)
mse_cost_function = torch.nn.MSELoss()
model_save_path = 'model_weights_testing_CHIGGS'
os.makedirs(model_save_path, exist_ok=True)
losses = []

print(v1_model)

CUDA is available! Training on GPU.
ModelV1(
  (layers): Sequential(
    (0): Linear(in_features=2, out_features=64, bias=True)
    (1): Tanh()
    (2): Linear(in_features=64, out_features=64, bias=True)
    (3): Tanh()
    (4): Linear(in_features=64, out_features=64, bias=True)
    (5): Tanh()
  )
  (final): Linear(in_features=64, out_features=1, bias=True)
)


In [ ]:
from tqdm import tqdm
import torch

num_phases = 5
epochs_per_phase = 5000
num_samples = 1000

for phase in range(num_phases):
    # Adjust the range for x_n and t_n based on the current phase
    lower_bound = phase * 0.2
    upper_bound = (phase + 1) * 0.2
    
    for epoch in tqdm(range(epochs_per_phase),
                      desc=f'Phase {phase+1} Progress:',  # Description includes current phase
                      leave=False,  # Do not leave the progress bar when done
                      ncols=75,  # Width of the progress bar
                      mininterval=0.1,
                      bar_format='{l_bar}{bar}|{remaining}',  # Only show the bar without any counters
                      colour='blue'):
        # Adjust the sampling of x_n and t_n within the new range
        x_n = (torch.rand(num_samples, 1) * (upper_bound - lower_bound) + lower_bound).to(device)
        t_n = (torch.rand(num_samples, 1) * (upper_bound - lower_bound) + lower_bound).to(device)
        
        optimizer.zero_grad()
        physics_loss_ur, physics_loss_ui, physics_loss_v = compute_physics_loss(u1_real_model, u1_imag_model, v1_model, x_n, t_n, mse_cost_function)
        domain_loss_ur_t, domain_loss_ui_t, domain_loss_v_t = compute_analytical_boundary_loss(u1_real_model, u1_imag_model, v1_model, x_n, t_n, mse_cost_function, k, omega, r)
        loss_u = physics_loss_ur + physics_loss_ui + domain_loss_ur_t + domain_loss_ui_t
        loss_v = physics_loss_v + domain_loss_v_t

        total_loss = loss_u + loss_v 
        total_loss.backward()
        
        # Print loss every few epochs within each phase
        if epoch % 1000 == 0 or epoch == epochs_per_phase - 1:  # Also print on the last epoch of the phase
            print(f'Phase {phase+1}, Epoch {epoch}, Loss U: {loss_u.item()}, Loss V: {loss_v.item()}, Total Loss: {total_loss.item()}')
        
model_filename = os.path.join(model_save_path, f'C_HIGGS_shared_1.pth')
torch.save(model.state_dict(), model_filename)

In [ ]:
model_path = os.path.join(model_save_path, 'C_HIGGS_shared_1.pth')
model = CombinedModel(input_dim=2, shared_units=128, branch_units_u=64, branch_units_v=32, output_dim=1, shared_layers=2, branch_layers_u=4, branch_layers_v=2, activation=nn.Tanh).to(device)
model.load_state_dict(torch.load(model_path))
model.train().to(device)

num_epochs = 100000  # Number of training epochs
lr = 1e-3          # Learning rate
num_samples = 1000 # Number of samples for training
r = 1.1
omega = 2
k = 0.5
factor = 1
lambda_ = 1e-3

optimizer = Adam(model.parameters(), lr=lr)

for epoch in tqdm(range(num_epochs),
                  desc='Progress:',  # Empty description
                  leave=False,  # Do not leave the progress bar when done
                  ncols=75,  # Width of the progress bar
                  mininterval=0.1,
                  bar_format='{l_bar}{bar}|{remaining}',  # Only show the bar without any counters
                  colour='blue'):
    x_n = (torch.rand(num_samples, 1)*factor).to(device)  # x in range [-5, -3]
    t_n = (torch.rand(num_samples, 1)*factor).to(device)   
    x_bc_x0 = torch.zeros((num_samples, 1)).to(device)
    t_bc_x0 = torch.rand((num_samples, 1)).to(device)  # Uniformly distributed random values between 0 and 1
    x_bc_x1 = torch.ones((num_samples, 1)).to(device)
    t_bc_x1 = torch.rand((num_samples, 1)).to(device)  # Uniformly distributed random values between 0 and 1
    x_bc_t0 = torch.rand((num_samples, 1)).to(device)  # Uniformly distributed random values between 0 and 1
    t_bc_t0 = torch.zeros((num_samples, 1)).to(device)
    x_bc_t1 = torch.rand((num_samples, 1)).to(device)  # Uniformly distributed random values between 0 and 1
    t_bc_t1 = torch.ones((num_samples, 1)).to(device)
    

    optimizer.zero_grad()
    physics_loss_ur, physics_loss_ui, physics_loss_v = compute_physics_loss(model, x_n, t_n, mse_cost_function)
    boundary_loss_ur_x0, boundary_loss_ui_x0, boundary_loss_v_x0 = compute_analytical_boundary_loss(model, x_bc_x0, t_bc_x0, mse_cost_function, k, omega, r)
    boundary_loss_ur_x1, boundary_loss_ui_x1, boundary_loss_v_x1 = compute_analytical_boundary_loss(model, x_bc_x1, t_bc_x1, mse_cost_function, k, omega, r)
    boundary_loss_ur_t0, boundary_loss_ui_t0, boundary_loss_v_t0 = compute_analytical_boundary_loss(model, x_bc_t0, t_bc_t0, mse_cost_function, k, omega, r)
    #boundary_loss_ur_t1, boundary_loss_ui_t1, boundary_loss_v_t1 = compute_analytical_boundary_loss(model_u1, model_v1, x_bc_t1, t_bc_t1, mse_cost_function, k, omega, r)
    domain_loss_ur_t, domain_loss_ui_t, domain_loss_v_t = compute_analytical_boundary_loss(model, x_n, t_n, mse_cost_function, k, omega, r)

    loss_u = lambda_*(physics_loss_ur + physics_loss_ui + domain_loss_ur_t + domain_loss_ui_t) + (1-lambda_)*(boundary_loss_ur_x0 + boundary_loss_ui_x0 + boundary_loss_ur_x1 + boundary_loss_ui_x1 + boundary_loss_ur_t0 + boundary_loss_ui_t0)
    loss_v = lambda_*(physics_loss_v + domain_loss_v_t) + (1-lambda_)*(boundary_loss_v_x0 + boundary_loss_v_x1 + boundary_loss_v_t0)
    total_loss = loss_u + loss_v 
    
    total_loss = loss_u + loss_v 
    total_loss.backward()
    # Print loss every few epochs
    if epoch % 1000 == 0:
        print(f'Epoch {epoch}, Loss U: {loss_u.item()}, Loss V: {loss_v.item()}')
        model_filename = os.path.join(model_save_path, f'C_HIGGS_shared_epoch_{epoch}.pth')
        torch.save(model.state_dict(), model_filename)
        
        df_losses = pd.DataFrame(losses)
        csv_file_path = 'loss_data/C_HIGGS_training_losses.csv'
        df_losses.to_csv(csv_file_path, index=False)
    
    if total_loss.item() < 1e-4:
        print(f'Stopping early at epoch {epoch} due to reaching target loss.')
        model_filename = os.path.join(model_save_path, f'C_HIGGS_shared_epoch_{epoch}.pth')
        torch.save(model.state_dict(), model_filename)
        break
    
    losses.append({
        'Epoch': epoch,
        'Loss U': loss_u.item(),
        'Loss V': loss_v.item(),
        'Total Loss': total_loss.item(),
        #'Physics Loss': physics_loss_u.item() + physics_loss_v.item(),
        #'Boundary Loss U': boundary_loss_u_x0.item() + boundary_loss_u_x1.item() + boundary_loss_u_t0.item() + boundary_loss_u_t1.item(),
        #'Boundary Loss V': boundary_loss_v_x0.item() + boundary_loss_v_x1.item() + boundary_loss_v_t0.item() + boundary_loss_v_t1.item(),
        #'Domain Loss U': domain_loss_u_t.item(),
        #'Domain Loss V': domain_loss_v_t.item()
    })

model_filename = os.path.join(model_save_path, f'C_HIGGS_shared_final.pth')
torch.save(model.state_dict(), model_filename)
        

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Generate a grid of x and t values
x = torch.linspace(0, 1, 400)  # More points for a smoother plot
t = torch.linspace(0, 1, 400)
X, T = torch.meshgrid(x, t)
X_flat = X.flatten()
T_flat = T.flatten()
omega = 1
#epoch = 21000

# Convert to torch tensors and prepare for the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
x_t = torch.stack([X_flat, T_flat], dim=1).to(device).float()

# Assuming shared_model is defined and loaded with trained parameters
model_u1_state = torch.load(os.path.join(model_save_path, f'C_HIGGS_U1_epoch_{epoch}.pth'), map_location=device)
model_v1_state = torch.load(os.path.join(model_save_path, f'C_HIGGS_V1_epoch_{epoch}.pth'), map_location=device)
model_u1.load_state_dict(model_u1_state)
model_v1.load_state_dict(model_v1_state)
model_u1.eval()
model_v1.eval()
# Get predictions from the trained models
with torch.no_grad():
    pred_u_r, pred_u_i = model_u1(x_t)
    pred_v = model_v1(x_t)

# Calculate the analytical solutions
real_u1_analytical = real_u1(X, T, k, omega, r)
imag_u1_analytical = imag_u1(X, T, k, omega, r)
real_v1_analytical = real_v1(X, T, k, omega, r)

# Convert predictions and analytical solutions back to NumPy for plotting
pred_u_r = pred_u_r.cpu().numpy().reshape(X.shape)
pred_u_i = pred_u_i.cpu().numpy().reshape(X.shape)
pred_v = pred_v.cpu().numpy().reshape(X.shape)
real_u1_analytical = real_u1_analytical.cpu().numpy().reshape(X.shape)
imag_u1_analytical = imag_u1_analytical.cpu().numpy().reshape(X.shape)
real_v1_analytical = real_v1_analytical.cpu().numpy().reshape(X.shape)


# Plotting
fig = plt.figure(figsize=(20, 6))

# Plot predicted and analytical real part of u1
ax1 = fig.add_subplot(131, projection='3d')
surf1 = ax1.plot_surface(X, T, pred_u_r, cmap='viridis')
fig.colorbar(surf1, ax=ax1, shrink=0.5, aspect=5)
ax1.set_title('Predicted Real Part of $u_1(x, t)$')
ax1.set_xlabel('x')
ax1.set_ylabel('t')
ax1.set_zlabel('Real part of $u_1$')

# Plot predicted and analytical imaginary part of u1
ax2 = fig.add_subplot(132, projection='3d')
surf2 = ax2.plot_surface(X, T, pred_u_i, cmap='viridis')
fig.colorbar(surf2, ax=ax2, shrink=0.5, aspect=5)
ax2.set_title('Predicted Imaginary Part of $u_1(x, t)$')
ax2.set_xlabel('x')
ax2.set_ylabel('t')
ax2.set_zlabel('Imag part of $u_1$')

# Plot predicted and analytical real part of v1
ax3 = fig.add_subplot(133, projection='3d')
surf3 = ax3.plot_surface(X, T, pred_v, cmap='viridis')
fig.colorbar(surf3, ax=ax3, shrink=0.5, aspect=5)
ax3.set_title('Predicted Real Part of $v_1(x, t)$')
ax3.set_xlabel('x')
ax3.set_ylabel('t')
ax3.set_zlabel('Real part of $v_1$')

plt.tight_layout()
plt.show()

# Plotting
fig = plt.figure(figsize=(20, 6))

# Plot predicted and analytical real part of u1
ax1 = fig.add_subplot(131, projection='3d')
surf1 = ax1.plot_surface(X, T, real_u1_analytical, cmap='viridis')
fig.colorbar(surf1, ax=ax1, shrink=0.5, aspect=5)
ax1.set_title('Analytical Real Part of $u_1(x, t)$')
ax1.set_xlabel('x')
ax1.set_ylabel('t')
ax1.set_zlabel('Real part of $u_1$')

# Plot predicted and analytical imaginary part of u1
ax2 = fig.add_subplot(132, projection='3d')
surf2 = ax2.plot_surface(X, T, imag_u1_analytical, cmap='viridis')
fig.colorbar(surf2, ax=ax2, shrink=0.5, aspect=5)
ax2.set_title('Analytical Imaginary Part of $u_1(x, t)$')
ax2.set_xlabel('x')
ax2.set_ylabel('t')
ax2.set_zlabel('Imag part of $u_1$')

# Plot predicted and analytical real part of v1
ax3 = fig.add_subplot(133, projection='3d')
surf3 = ax3.plot_surface(X, T, real_v1_analytical, cmap='viridis')
fig.colorbar(surf3, ax=ax3, shrink=0.5, aspect=5)
ax3.set_title('Analytical Real Part of $v_1(x, t)$')
ax3.set_xlabel('x')
ax3.set_ylabel('t')
ax3.set_zlabel('Real part of $v_1$')

plt.tight_layout()
plt.show()

